In [1]:
import ast
import pandas as pd 
from sklearn.model_selection import train_test_split

# Read Raw Data

> Raw data are downloaded from Kaggle [Game Recommendations on Steam](https://www.kaggle.com/datasets/antonkozyriev/game-recommendations-on-steam).
> 
> There are four files:
> 1. `games.csv`: Contains games data in table format.
> 2. `games_metadata.json`: Contains games metadata, which can be viewed as additional data for the games table. This file is in JSON format.
> 3. `recommendations.csv`: Contains recommendation records, which is the main data table. This is in table format.
> 4. `users.csv`: Contains user data in table format.

In [2]:
games_df = pd.read_csv("../data/games.csv")
games_meta_df = pd.read_json("../data/games_metadata.json", lines=True)
recommendations_df = pd.read_csv("../data/recommendations.csv")
users_df = pd.read_csv("../data/users.csv")

# Explore all four tables

In [3]:
print(games_df.shape)
games_df.head()


(50872, 13)


,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck
0,13500,Prince of Persia: Warrior Within™,2008-11-21,True,False,False,Very Positive,84,2199,9.99,9.99,0.0,True
1,22364,BRINK: Agents of Change,2011-08-03,True,False,False,Positive,85,21,2.99,2.99,0.0,True
2,113020,Monaco: What's Yours Is Mine,2013-04-24,True,True,True,Very Positive,92,3722,14.99,14.99,0.0,True
3,226560,Escape Dead Island,2014-11-18,True,False,False,Mixed,61,873,14.99,14.99,0.0,True
4,249050,Dungeon of the ENDLESS™,2014-10-27,True,True,False,Very Positive,88,8784,11.99,11.99,0.0,True


In [4]:
print(games_meta_df.shape)
games_meta_df.head()

(50872, 3)


,app_id,description,tags
0,13500,Enter the dark underworld of Prince of Persia ...,"[Action, Adventure, Parkour, Third Person, Gre..."
1,22364,,[Action]
2,113020,Monaco: What's Yours Is Mine is a single playe...,"[Co-op, Stealth, Indie, Heist, Local Co-Op, St..."
3,226560,Escape Dead Island is a Survival-Mystery adven...,"[Zombies, Adventure, Survival, Action, Third P..."
4,249050,Dungeon of the Endless is a Rogue-Like Dungeon...,"[Roguelike, Strategy, Tower Defense, Pixel Gra..."


In [5]:
print(recommendations_df.shape)
recommendations_df.head()

(41154794, 8)


,app_id,helpful,funny,date,is_recommended,hours,user_id,review_id
0,975370,0,0,2022-12-12,True,36.3,51580,0
1,304390,4,0,2017-02-17,False,11.5,2586,1
2,1085660,2,0,2019-11-17,True,336.5,253880,2
3,703080,0,0,2022-09-23,True,27.4,259432,3
4,526870,0,0,2021-01-10,True,7.9,23869,4


In [6]:
print(users_df.shape)
users_df.head()

(14306064, 3)


,user_id,products,reviews
0,7360263,359,0
1,14020781,156,1
2,8762579,329,4
3,4820647,176,4
4,5167327,98,2


# Data Sampling
> Recommendations data is too large for us to process; we need to sample it to 1 million.

In [7]:
recommendations_100000_samples_df = recommendations_df.sample(100000, random_state=2024)
# recommendations_100000_samples_df.to_csv("../data/recommendations_100000_samples.csv", index=False)
recommendations_100000_samples_df

,app_id,helpful,funny,date,is_recommended,hours,user_id,review_id
37846045,1986230,3,2,2022-11-24,True,1.2,9199981,37846045
34534027,203160,0,0,2020-03-31,True,10.2,8830375,34534027
24857859,460920,0,0,2019-03-14,True,60.6,9601354,24857859
31367452,1291010,0,0,2021-11-25,True,1.5,7206687,31367452
16967846,1089980,0,0,2020-08-26,True,6.3,11148502,16967846
...,...,...,...,...,...,...,...,...
3689402,284160,0,0,2022-11-14,True,52.9,13540086,3689402
16228781,282900,0,0,2016-11-24,True,102.1,11764785,16228781
32318785,314410,0,0,2020-12-05,True,88.7,13080604,32318785
35098803,274940,0,0,2018-11-01,True,2.7,9266366,35098803


# Data Merging

> The Recommendations table is the primary dataset. We should consider it as the main dataframe and join other tables into this table.
> 
> - For games and games meta data, we can join them to the recommendations table on `app_id`.
> - For user data, we can join them to recommendations on `user_id`.
> 
> After doing this, we will have one comprehensive dataframe that has all the related features. We will do further data preprocessing on this dataframe later.

In [8]:
df = recommendations_100000_samples_df.merge(
    games_df, how="left", on="app_id"
).merge(
    games_meta_df, how="left", on="app_id"
).merge(
    users_df, how="left", on="user_id"
)
df.head(1)

,app_id,helpful,funny,date,is_recommended,hours,user_id,review_id,title,date_release,...,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck,description,tags,products,reviews
0,1986230,3,2,2022-11-24,True,1.2,9199981,37846045,What if your girl was a frog?,2022-06-03,...,90,157,0.99,0.99,0.0,True,There was a frog. She was cheerfully croaking ...,"[Sexual Content, Nudity, Mature, Anime, Casual...",452,5


# Data Flattening
> `tags` is a very tricky column, we should creat dummy variables by unnesting tags column

In [9]:
df["tags"]

0        [Sexual Content, Nudity, Mature, Anime, Casual...
1        [Adventure, Action, Female Protagonist, Third ...
2        [Sports, Open World, Multiplayer, Action, Simu...
3        [Strategy, Tower Defense, RTS, War, Resource M...
4                                                       []
                               ...                        
99995                                                   []
99996    [Anime, JRPG, Female Protagonist, Cute, RPG, S...
99997    [Co-op, Action Roguelike, Action, Beat 'em up,...
99998    [Action, Multiplayer, Underwater, Gore, Team-B...
99999    [Psychological Horror, Multiplayer, Free to Pl...
Name: tags, Length: 100000, dtype: object

In [10]:
##### Convert the type of tags from string to list in Python.
##### df["tags"] = df.tags.apply(ast.literal_eval)

# Explode the 'Tags' column into separate rows
df_tags_expanded = df.explode('tags')
# Create dummy variables for each tag
dummy_df = pd.get_dummies(df_tags_expanded['tags'], prefix="tag")
# Group by the original index and aggregate the dummy variables
grouped_dummy_df = dummy_df.groupby(dummy_df.index).sum()
# Concatenate the original DataFrame with the dummy variables, remove old `tags` column
df = pd.concat([df.drop(columns='tags'), grouped_dummy_df], axis=1)
df

,app_id,helpful,funny,date,is_recommended,hours,user_id,review_id,title,date_release,...,tag_Well-Written,tag_Werewolves,tag_Western,tag_Wholesome,tag_Word Game,tag_World War I,tag_World War II,tag_Wrestling,tag_Zombies,tag_eSports
0,1986230,3,2,2022-11-24,True,1.2,9199981,37846045,What if your girl was a frog?,2022-06-03,...,0,0,0,0,0,0,0,0,0,0
1,203160,0,0,2020-03-31,True,10.2,8830375,34534027,Tomb Raider,2013-03-04,...,0,0,0,0,0,0,0,0,0,0
2,460920,0,0,2019-03-14,True,60.6,9601354,24857859,Steep™,2016-12-02,...,0,0,0,0,0,0,0,0,0,0
3,1291010,0,0,2021-11-25,True,1.5,7206687,31367452,Warpips,2022-04-21,...,0,0,0,0,0,0,0,0,0,0
4,1089980,0,0,2020-08-26,True,6.3,11148502,16967846,The Henry Stickmin Collection,2020-08-07,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,284160,0,0,2022-11-14,True,52.9,13540086,3689402,BeamNG.drive,2015-05-29,...,0,0,0,0,0,0,0,0,0,0
99996,282900,0,0,2016-11-24,True,102.1,11764785,16228781,Hyperdimension Neptunia Re;Birth1,2015-01-29,...,0,0,0,0,0,0,0,0,0,0
99997,314410,0,0,2020-12-05,True,88.7,13080604,32318785,Rampage Knights,2015-09-04,...,0,0,0,0,0,0,0,0,0,0
99998,274940,0,0,2018-11-01,True,2.7,9266366,35098803,Depth,2014-11-03,...,0,0,0,0,0,0,0,0,0,0


# Data Splitting

> To prevent data leakage, we split the data before doing EDA and preprocessing.

In [11]:
train_df, test_df = train_test_split(
    df, 
    test_size=0.2,
    random_state=2024,
)

In [12]:
# This is a huge file, save it in gzip compressed format.
df.to_csv("../data/1-recommendations_merged_100000_samples-all.csv.gz", index=False, compression="gzip")
train_df.to_csv("../data/1-recommendations_merged_100000_samples-train.csv.gz", index=False, compression="gzip")
test_df.to_csv("../data/1-recommendations_merged_100000_samples-test.csv.gz", index=False, compression="gzip")